#### TLS 24

In [ ]:
# 1. load data
import os
import pandas as pd
import src.utils as ut
import prompt_builder as pb

from dotenv import load_dotenv
from src.data_loader import load_and_prepare_projects, load_test_dataset
from prompt_utils import tokens_counter, prompt_cost


path_22 = 'data/tls22_projects.csv'
path_24 = 'data/tls24_projects.csv'

df, df22_shuffled = load_and_prepare_projects(path_22,path_24, city = "Toulouse")
test = load_test_dataset(df, rows = len(df[df['year'] == 2024]), city = 'Toulouse')

load_dotenv()

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

In [ ]:
#2. build prompt
top_k_voted_22 = pb.get_top_k_projects_from_22_election(df, k=15)
proj_count_avg_by_district= pb.get_project_count_and_avg_votes_by_district(df)
proj_count_avg_by_categories = pb.get_project_count_and_avg_votes_by_categories(df)

test['prompt'] = test.apply(
    lambda x: pb.build_prompt(
        'prompts/prompt_rag2_SB.txt',
        {
            'top_k_voted_22': top_k_voted_22,
            'project_count_and_avg_votes_by_district': proj_count_avg_by_district,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description'],
            'top_k_similar_projects_in_22': pb.get_top_k_similar_projects_in_22(df,x['project_id'],conn_params, k=10),
            'count_of_projects22_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district']),
            'top_k_voted_in_district': pb.get_top_k_voted_in_district(df, x['district'], k=5),
            'count_of_projects24_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2024),
            'top_k_similar_projects_in_district': pb.get_top_k_similar_projects_in_22_by_district(df, x['project_id'], conn_params,k=5)
        }
    ),
    axis=1
)


In [ ]:
print(test['prompt'].iloc[0])

In [ ]:
test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('avg.cost of each prediction: ${:.2f}'.format(test.cost_usd.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))

In [ ]:
#3. run experiment! :)
api_key=os.getenv('OPENAI_API_KEY')
from llm_client import call_openai_model 

test['out'] = test['prompt'].apply(lambda prompt: 
                     call_openai_model(prompt=prompt, api_key=api_key))

In [ ]:
import json
import re

def get_json_from_llm_response(out):
    
    if isinstance(out, str):
        try:
            data = json.loads(out)
            if isinstance(data, dict):
                return data
        except json.JSONDecodeError:
            pass

    match = re.search(r"```json\s*({.*?})\s*```", out, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None

    return None

def get_predicted_votes(df, out_json, city = 'Toulouse'):
    
    if city != 'Toulouse':
        raise ValueError(f"City '{city}' not supported")

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
    
    votes_2022 = df[df['year'] == 2022].votes.sum()
    votes_2024 = df[df['year'] == 2024].votes.sum()

    relative_estimation = out_json['voix_estimées'] / votes_2022
    adjusted_prediction = relative_estimation * votes_2024

    return adjusted_prediction

def get_predicted_rank(out_json):
    
    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['position_attendue']


def get_if_is_top5(out_json):

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['dans_top_5']

def get_if_is_top10(out_json):

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['dans_top_10']

In [ ]:
test['out_json'] = test['out'].apply(
    lambda o: get_json_from_llm_response(o))

In [ ]:
test['out_json'].iloc[0]

In [ ]:
test['predicted_votes'] = test['out_json'].apply(lambda x: get_predicted_votes(df, x))
test['predicted_rank'] = test['out_json'].apply(lambda x: get_predicted_rank(x))
test['is_top5'] = test['out_json'].apply(lambda x: get_if_is_top5(x))
test['is_top10'] = test['out_json'].apply(lambda x: get_if_is_top10(x))

In [ ]:
#4. save results.
results = test.filter(['project_id', 
             'real_votes', 
             'real_rank',
             'prompt',
             'out',
             'out_json', 
             'predicted_votes', 
             'predicted_rank', 
             'is_top5', 
             'is_top10']
             )
results

In [ ]:
results.to_csv('output/predictions/tls_rag_sb_full_1.csv', sep=";", index=False)
print('results ready!')

### TLS 24 RAG SB - Eng DB

In [1]:
# 1. load data
import os
import pandas as pd
import src.utils as ut
import prompt_builder as pb

from dotenv import load_dotenv
from src.data_loader import load_and_prepare_projects, load_test_dataset
from prompt_utils import tokens_counter, prompt_cost

load_dotenv()

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

path_22 = 'data/tls22_projects.csv'
path_24 = 'data/tls24_projects.csv'

df, df22_shuffled = load_and_prepare_projects(path_22,path_24, city = "Toulouse")
test = load_test_dataset(df, rows = len(df[df['year'] == 2024]), city = 'Toulouse')

english_df  = pd.read_csv("data/tls_projects_translated_to_eng.csv", sep=";")

df = pd.merge(
    left=df,
    right=english_df,
    on=['project_id','year'],
    how='inner'
)

test = pd.merge(left=test,
         right=english_df,
         on=['project_id'],
         how='inner')

In [2]:
#2. build prompt
top_k_voted_22 = pb.get_top_k_projects_from_22_election_eng(df, k=15)
proj_count_avg_by_district= pb.get_project_count_and_avg_votes_by_district(df)
proj_count_avg_by_categories = pb.get_project_count_and_avg_votes_by_categories(df)

test['prompt'] = test.apply(
    lambda x: pb.build_prompt(
        'prompts/tls_prompt_rag_SB_eng.txt',
        {
            'top_k_voted_22': top_k_voted_22,
            'project_count_and_avg_votes_by_district': proj_count_avg_by_district,
            'project_name': x['project_name_eng'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description_eng'],
            'top_k_similar_projects_in_22': pb.get_top_k_similar_projects_in_22_eng(df,x['project_id'],conn_params, k=10),
            'count_of_projects22_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district']),
            'top_k_voted_in_district': pb.get_top_k_voted_in_district_eng(df, x['district'], k=5),
            'count_of_projects24_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2024),
            'top_k_similar_projects_in_district': pb.get_top_k_similar_projects_in_22_by_district_eng(df, x['project_id'], conn_params,k=5)
        }
    ),
    axis=1
)

In [3]:
print(test['prompt'].iloc[0])

You are an expert model in analyzing participatory budgeting elections, specifically in the context of the city of Toulouse, France.

In Toulouse, the municipality organizes participatory budgeting elections to fund citizen-led projects. Each resident can vote for between 1 and 3 initiatives via the official website. Votes are anonymous.

Here is how projects are selected:
- All projects are ranked according to the number of votes received.
- A greedy algorithm is applied: projects are funded in decreasing order of votes, as long as the total budget is not exceeded.
- If a project is too expensive for the remaining budget, it is skipped and the algorithm proceeds to the next one.

---

In the 2022 election:
- There were 4,532 voters, each allowed to vote for up to 3 projects.
- A total of approximately 11,918 votes were cast across 200 projects.
- The number of votes per project ranged from 2 to 492, with an average around 58 votes and an estimated standard deviation of 68.
- Most proj

In [4]:
test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('avg.cost of each prediction: ${:.2f}'.format(test.cost_usd.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))

mean tokens by prompt: 3591.07
avg.cost of each prediction: $0.04
experiment total cost: 6.57


In [ ]:
#3. run experiment! :)
api_key=os.getenv('OPENAI_API_KEY')
from llm_client import call_openai_model 

test['out'] = test['prompt'].apply(lambda prompt: 
                     call_openai_model(prompt=prompt, api_key=api_key))

In [ ]:
import json
import re

def get_json_from_llm_response(out):
    
    if isinstance(out, str):
        try:
            data = json.loads(out)
            if isinstance(data, dict):
                return data
        except json.JSONDecodeError:
            pass

    match = re.search(r"```json\s*({.*?})\s*```", out, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None

    return None

def get_predicted_votes(df, out_json, city = 'Toulouse'):
    
    if city != 'Toulouse':
        raise ValueError(f"City '{city}' not supported")

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
    
    votes_2022 = df[df['year'] == 2022].votes.sum()
    votes_2024 = df[df['year'] == 2024].votes.sum()

    relative_estimation = out_json['estimated_votes'] / votes_2022
    adjusted_prediction = relative_estimation * votes_2024

    return adjusted_prediction

def get_predicted_rank(out_json):
    
    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['expected_position']


def get_if_is_top5(out_json):

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['likely_in_top_5']

def get_if_is_top10(out_json):

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['likely_in_top_10']

In [ ]:
test['out_json'] = test['out'].apply(
    lambda o: get_json_from_llm_response(o))

In [ ]:
test['out_json'].iloc[0]

In [ ]:
test['predicted_votes'] = test['out_json'].apply(lambda x: get_predicted_votes(df, x))
test['predicted_rank'] = test['out_json'].apply(lambda x: get_predicted_rank(x))
test['is_top5'] = test['out_json'].apply(lambda x: get_if_is_top5(x))
test['is_top10'] = test['out_json'].apply(lambda x: get_if_is_top10(x))

In [ ]:
#4. save results.
results = test.filter(['project_id', 
             'real_votes', 
             'real_rank',
             'prompt',
             'out',
             'out_json', 
             'predicted_votes', 
             'predicted_rank', 
             'is_top5', 
             'is_top10']
             )
results

In [ ]:
results.to_csv('output/predictions/tls_rag_sb_full_eng_1.csv', sep=";", index=False)
print('results ready!')

### WRC 17 SB

In [1]:
# 1. load data
import os
import pandas as pd
import src.utils as ut
import prompt_builder as pb

from dotenv import load_dotenv
from src.data_loader import load_and_prepare_projects, load_test_dataset
from prompt_utils import tokens_counter, prompt_cost


path_16 = 'data/wrc16_projects.csv'
path_17 = 'data/wrc17_projects.csv'

df, df16_shuffled = load_and_prepare_projects(path_16,path_17, city = 'Wroclaw')
test = load_test_dataset(df, rows = 50, city='Wroclaw')

load_dotenv()

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

test

,project_id,project_name,description,cost,district,district_number,real_votes,real_rank,year
0,10,Toalety we Wrocławskich parkach i na terenach ...,Uzasadnienie\nW parkach i na terenach zielonyc...,900000,Śródmieście,4.0,10857,1,2017
1,50,Zielona rowerowo-piesza obwodnica Wrocławia; E...,Uzasadnienie\nWalczymy o II ETAP. Ideą jest po...,1000000,Stare Miasto,5.0,10796,2,2017
2,18,Rowerowy Wrocław 2017,Uzasadnienie\nProjekt przewiduje poprawę bezpi...,1000000,Stare Miasto,5.0,8640,3,2017
3,675,"Zieleń dla Wrocławia – parki kieszonkowe, nasa...",Uzasadnienie\nWe Wrocławiu nastąpiła deklaraty...,1000000,Psie Pole,3.0,5398,4,2017
4,260,Budki dla kotów wolno żyjących,Uzasadnienie,70000,Psie Pole,3.0,4998,5,2017
5,550,Pieszo z Brochowa - uzupełnienie brakującego c...,Uzasadnienie\nZ Brochowa nie sposób wydostać s...,1000000,Krzyki,2.0,4468,6,2017
6,12,Bulwar Fizyków - naukowy plac zabaw w przestrz...,Uzasadnienie\nProponujemy stworzenie instalacj...,1000000,Psie Pole,3.0,4149,7,2017
7,159,Baza małych astronautów czyli kosmiczny plac z...,Uzasadnienie\nW ramach Budżetu Obywatelskiego ...,1000000,Krzyki,2.0,4098,8,2017
8,656,Toalety miejskie w parkach na miarę Wrocławia.,Uzasadnienie\nInicjatywa jest odpowiedzią na p...,1000000,Śródmieście,4.0,4008,9,2017
9,499,"""Konikowo"" - Budowa autorskiego placu zabaw na...","Uzasadnienie\nDrodzy Wrocławianie, Drodzy Sąsi...",1000000,Stare Miasto,5.0,3949,10,2017


In [2]:
conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

#2. build prompt
top_k_voted_16 = pb.get_top_k_projects_from_16_election(df, k=15)
proj_count_avg_by_district= pb.get_project_count_and_avg_votes_by_district(df, city = 'Wroclaw')
proj_count_avg_by_categories = pb.get_project_count_and_avg_votes_by_categories(df, city = 'Wroclaw')

In [4]:
test['prompt'] = test.apply(
    lambda x: pb.build_prompt(
        'prompts/prompt_rag2_SB_pl.txt',
        {
            'top_k_voted_16': top_k_voted_16,
            'project_count_and_avg_votes_by_district': proj_count_avg_by_district,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description'],
            'top_k_similar_projects_in_16': pb.get_top_k_similar_projects_in_16(df, x['project_id'], x['year'], conn_params,k=5),
            'count_of_projects16_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2016),
            'top_k_voted_in_district': pb.get_top_k_voted_in_district_in_wro(df, x['district'], k=5, year=2016),
            'count_of_projects17_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2017),
            'top_k_similar_projects_in_district': pb.get_top_k_similar_projects_in_16_by_district(df, x['project_id'], x['year'], conn_params,k=5)
        }
    ),
    axis=1
)

In [5]:
print(test['prompt'].iloc[0])

You are an expert model in the analysis of participatory budgeting elections, specifically in the context of the city of Wroclaw, Poland.

In Wroclaw, the municipality organizes participatory budgeting elections to fund citizen-led projects. Each resident can vote for between 1 and 3 initiatives on the official website.
Votes are anonymous.

Here is how the projects are selected:
- All projects are ranked by the number of votes received.
- A "greedy" algorithm is used: projects are funded in descending order of votes, as long as the total budget is not exceeded.
- If a project is too expensive for the remaining budget, it is skipped and the next one is considered.

---
In the 2016 election:
- There were **67103 voters**, each allowed to vote for up to 3 projects.
- In total, approximately **119194 votes** were cast across **52 projets**.
- The number of votes per project ranged from **120** to **13938**, with an average of **2292.19** votes and an estimated standard deviation of **2784

In [6]:
test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('avg.cost of each prediction: ${:.2f}'.format(test.cost_usd.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))

mean tokens by prompt: 3014.36
avg.cost of each prediction: $0.03
experiment total cost: 1.51


In [ ]:
#3. run experiment! :)
api_key=os.getenv('OPENAI_API_KEY')
from llm_client import call_openai_model 

test['out'] = test['prompt'].apply(lambda prompt: 
                     call_openai_model(prompt=prompt, api_key=api_key))

In [ ]:
import json
import re

def get_json_from_llm_response(out):
    # 1. Primer intento: el string completo es un JSON válido
    if isinstance(out, str):
        try:
            data = json.loads(out)
            if isinstance(data, dict):
                return data
        except json.JSONDecodeError:
            pass  # No es un JSON plano válido, continuamos

    # 2. Segundo intento: buscar bloque ```json {...} ```
    match = re.search(r"```json\s*({.*?})\s*```", out, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None

    # 3. No se pudo encontrar JSON válido
    return None

def get_predicted_votes(df, out_json, city = 'Wroclaw'):
    
    if city != 'Wroclaw':
        raise ValueError(f"City '{city}' not supported")

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
    
    votes_2016 = df[df['year'] == 2016].votes.sum()
    votes_2017 = df[df['year'] == 2017].votes.sum()

    relative_estimation = out_json['estimated_votes'] / votes_2016
    adjusted_prediction = relative_estimation * votes_2017

    return adjusted_prediction

def get_predicted_rank(out_json):
    
    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['expected_rank']

def get_if_is_top5(out_json):
    return out_json['in_top_5']

def get_if_is_top10(out_json):
    return out_json['in_top_10']


In [ ]:
test['out_json'] = test['out'].apply(
    lambda o: get_json_from_llm_response(o))

In [ ]:
test['predicted_votes'] = test['out_json'].apply(lambda x: get_predicted_votes(df, x))
test['predicted_rank'] = test['out_json'].apply(lambda x: get_predicted_rank(x))
test['is_top5'] = test['out_json'].apply(lambda x: get_if_is_top5(x))
test['is_top10'] = test['out_json'].apply(lambda x: get_if_is_top10(x))

In [ ]:
#4. save results.
results = test.filter(['project_id', 
             'real_votes', 
             'real_rank',
             'prompt',
             'out',
             'out_json', 
             'predicted_votes', 
             'predicted_rank', 
             'is_top5', 
             'is_top10']
             )
results

In [ ]:
results.to_csv('output/predictions/wrc_rag_sb_full_1.csv', sep=";", index=False)
print('results ready!')

### WRC 17 Eng

In [7]:
# 1. load data
import os
import pandas as pd
import src.utils as ut
import prompt_builder as pb

from dotenv import load_dotenv
from src.data_loader import load_and_prepare_projects, load_test_dataset
from prompt_utils import tokens_counter, prompt_cost


df = pd.read_csv("data/wrc_projects_eng.csv", sep=";")
df16_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
test = df[df['year'] == 2017].sample(frac=1, random_state=42).reset_index(drop=True)


load_dotenv()

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

In [8]:
conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

#2. build prompt
top_k_voted_16 = pb.get_top_k_projects_from_16_election(df, k=15)
proj_count_avg_by_district= pb.get_project_count_and_avg_votes_by_district(df, city = 'Wroclaw')
proj_count_avg_by_categories = pb.get_project_count_and_avg_votes_by_categories(df, city = 'Wroclaw')

In [10]:
test['prompt'] = test.apply(
    lambda x: pb.build_prompt(
        'prompts/prompt_rag2_SB_pl.txt',
        {
            'top_k_voted_16': top_k_voted_16,
            'project_count_and_avg_votes_by_district': proj_count_avg_by_district,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description'],
            'top_k_similar_projects_in_16': pb.get_top_k_similar_projects_in_16(df, x['project_id'], x['year'], conn_params,k=5),
            'count_of_projects16_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2016),
            'top_k_voted_in_district': pb.get_top_k_voted_in_district_in_wro(df, x['district'], k=5, year=2016),
            'count_of_projects17_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2017),
            'top_k_similar_projects_in_district': pb.get_top_k_similar_projects_in_16_by_district(df, x['project_id'], x['year'], conn_params,k=5)
        }
    ),
    axis=1
)

In [11]:
print(test['prompt'].iloc[0])

You are an expert model in the analysis of participatory budgeting elections, specifically in the context of the city of Wroclaw, Poland.

In Wroclaw, the municipality organizes participatory budgeting elections to fund citizen-led projects. Each resident can vote for between 1 and 3 initiatives on the official website.
Votes are anonymous.

Here is how the projects are selected:
- All projects are ranked by the number of votes received.
- A "greedy" algorithm is used: projects are funded in descending order of votes, as long as the total budget is not exceeded.
- If a project is too expensive for the remaining budget, it is skipped and the next one is considered.

---
In the 2016 election:
- There were **67103 voters**, each allowed to vote for up to 3 projects.
- In total, approximately **119194 votes** were cast across **52 projets**.
- The number of votes per project ranged from **120** to **13938**, with an average of **2292.19** votes and an estimated standard deviation of **2784

In [12]:
test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('avg.cost of each prediction: ${:.2f}'.format(test.cost_usd.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))

mean tokens by prompt: 2536.70
avg.cost of each prediction: $0.03
experiment total cost: 1.27


In [ ]:
#3. run experiment! :)
api_key=os.getenv('OPENAI_API_KEY')
from llm_client import call_openai_model 

test['out'] = test['prompt'].apply(lambda prompt: 
                     call_openai_model(prompt=prompt, api_key=api_key))

In [ ]:
import json
import re

def get_json_from_llm_response(out):
    # 1. Primer intento: el string completo es un JSON válido
    if isinstance(out, str):
        try:
            data = json.loads(out)
            if isinstance(data, dict):
                return data
        except json.JSONDecodeError:
            pass  # No es un JSON plano válido, continuamos

    # 2. Segundo intento: buscar bloque ```json {...} ```
    match = re.search(r"```json\s*({.*?})\s*```", out, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None

    # 3. No se pudo encontrar JSON válido
    return None

def get_predicted_votes(df, out_json, city = 'Wroclaw'):
    
    if city != 'Wroclaw':
        raise ValueError(f"City '{city}' not supported")

    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
    
    votes_2016 = df[df['year'] == 2016].votes.sum()
    votes_2017 = df[df['year'] == 2017].votes.sum()

    relative_estimation = out_json['estimated_votes'] / votes_2016
    adjusted_prediction = relative_estimation * votes_2017

    return adjusted_prediction

def get_predicted_rank(out_json):
    
    if isinstance(out_json, str):
        try:
            out_json = json.loads(out_json)
        except json.JSONDecodeError:
            return None
        
    return out_json['expected_rank']

def get_if_is_top5(out_json):
    return out_json['in_top_5']

def get_if_is_top10(out_json):
    return out_json['in_top_10']


In [ ]:
test['out_json'] = test['out'].apply(
    lambda o: get_json_from_llm_response(o))

In [ ]:
test['predicted_votes'] = test['out_json'].apply(lambda x: get_predicted_votes(df, x))
test['predicted_rank'] = test['out_json'].apply(lambda x: get_predicted_rank(x))
test['is_top5'] = test['out_json'].apply(lambda x: get_if_is_top5(x))
test['is_top10'] = test['out_json'].apply(lambda x: get_if_is_top10(x))

In [ ]:
#4. save results.
results = test.filter(['project_id', 
             'votes', 
             'real_rank',
             'prompt',
             'out',
             'out_json', 
             'predicted_votes', 
             'predicted_rank', 
             'is_top5', 
             'is_top10']
             )
results

In [ ]:
results.to_csv('output/predictions/wrc_rag_sb_full_eng_1.csv', sep=";", index=False)
print('results ready!')